In [1]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
# from PIL import Image
import xml.etree.ElementTree as ET
# from io import BytesIO
# import dlib
# from xmltodict import parse
import _pickle as pickle
import sys
from csv import reader
from xml.dom import minidom
import math
import random

np.set_printoptions(threshold=sys.maxsize)

In [2]:
#get all annotations and image names
#extract last number
store_path = "data/data2"
images = [f for f in listdir(store_path+"/images") if isfile(join(store_path+"/images", f))]
next_num = len(images)
prefix = 'maksssksksss'

In [3]:
next_num

853

In [4]:
path_data3 = "data/data3/"
path_im3="data/data3/images/" 

In [5]:
df = pd.read_csv(path_data3+'train.csv')
data3_df = df[(df['classname'] == 'face_with_mask') | (df['classname'] == 'face_no_mask')]
print(data3_df.shape)
error_files = list(set(df[(df['y1'] == df['y2']) |  (df['x1'] == df['x2'])]['name']))
data3_df =  data3_df[~data3_df.name.isin(error_files)]
print(data3_df.shape)
data_3_group_by_name = data3_df.groupby("name")

# image3_df = data3_df.apply(lambda row: cv2.cvtColor(cv2.imread(os.path.join(path_im3,row['name'])),cv2.COLOR_BGR2RGB), axis=1)

# data3_df = data3_df.merge(image3_df.rename('image'), left_index=True, right_index=True)

(5749, 6)
(5610, 6)


In [6]:
print(data_3_group_by_name)

In [7]:
path_data1 = "data/data1/"
#get last in data2 and rename and add
path_im1_with="data/data1/with_mask/"
#get last in data2 and rename and add
path_im1_without="data/data1/without_mask/" 

In [8]:
def getBoundaryBoxPoints(img, objectList, net, label=None):
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(img, 1.0,(300, 300), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces = net.forward()
    #to draw faces on image
    rectangle_list = []
    for i in range(faces.shape[2]):
            objectDict = {}
            confidence = faces[0, 0, i, 2]
            if confidence > 0.76:
                box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x, y, x1, y1) = box.astype("int")
                (startX, startY) = (max(0, x), max(0, y))
                (endX, endY) = (min(w - 1, x1), min(h - 1, y1))
                
                objectDict['name'] = filename.rsplit('/', 1)[-1]
                objectDict['classname'] = label
                objectDict['x1'] = startX
                objectDict['y1'] = startY
                objectDict['x2'] = endX
                objectDict['y2'] = endY
#                 objectDict['image'] = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                objectList.append(dict(objectDict))
    

In [9]:
#TODO: change code later 
modelFile = "models/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "models/deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(path_data1+configFile, path_data1+modelFile)
objectList_with = []
objectList_without = []

for filename in os.listdir(path_im1_with):
    img = cv2.imread(path_im1_with+filename)
    if img is not None:
        getBoundaryBoxPoints(img, objectList_with, net, 'face_with_mask')
    
for filename in os.listdir(path_im1_without):
    img = cv2.imread(path_im1_without+filename)
    if img is not None:
        getBoundaryBoxPoints(img, objectList_without, net, 'face_no_mask')

In [10]:
data1_df_with = pd.DataFrame.from_records(objectList_with)
data_1_with_group_by_name = data1_df_with.groupby("name")
# data1_df.to_csv('data1.csv')

data1_df_without = pd.DataFrame.from_records(objectList_without)
data_1_without_group_by_name = data1_df_without.groupby("name")

In [11]:
def get_size(path,name):
    image = cv2.cvtColor(cv2.imread(os.path.join(path,name)),cv2.COLOR_BGR2RGB)
    return image.shape

In [12]:
def get_min_max(num1,num2):
    if num1 <= num2:
        return num1,num2
    else:
        return num2,num1

In [13]:
def get_image(path,name):
    image = cv2.imread(os.path.join(path,name))
    return image

In [14]:
def create_xml(store_path,file_path,newname,imagename,group):
    annotation = ET.Element("annotation") 
    
    folder = ET.SubElement(annotation, "folder") 
    folder.text = "images"
    filename = ET.SubElement(annotation, "filename") 
    filename.text = newname+".png"
    size = ET.SubElement(annotation, "size") 
    imagelength,imagewidth,imagechannel = get_size(file_path,imagename)
    width = ET.SubElement(size,"width")
    width.text = str(imagewidth)
    length = ET.SubElement(size,"height")
    length.text = str(imagelength)
    depth = ET.SubElement(size,"depth")
    depth.text = str(imagechannel)
    
    segmented = ET.SubElement(annotation,"segmented")
    segmented.text = '0'
        
    for i,row in group.iterrows():
        obj = ET.SubElement(annotation,"object")
        name = ET.SubElement(obj,"name")
        name.text = row['classname']
        pose = ET.SubElement(obj,"pose")
        pose.text = "none"
        truncated = ET.SubElement(obj,"truncated")
        truncated.text = "0"
        occluded = ET.SubElement(obj,"occluded")
        occluded.text = "0"
        difficult = ET.SubElement(obj,"difficult")
        difficult.text = "0"
        bndbox = ET.SubElement(obj,"bndbox")
        
        imagexmin,imagexmax = get_min_max(row['x1'],row['x2'])
        imageymin,imageymax = get_min_max(row['y1'],row['y2'])
        
        
        xmin = ET.SubElement(bndbox,"xmin")
        xmin.text = str(imagexmin)
        ymin = ET.SubElement(bndbox,"ymin")
        ymin.text = str(imageymin)
        xmax = ET.SubElement(bndbox,"xmax")
        xmax.text = str(imagexmax)
        ymax = ET.SubElement(bndbox,"ymax")
        ymax.text = str(imageymax)
        
    voc = ET.ElementTree(annotation) 
    image = get_image(file_path,imagename)
    with open(store_path+"/annotations/"+newname+".xml", "wb") as f : 
        voc.write(f) 
    
    cv2.imwrite(store_path+"/images/"+newname+'.png', image)
        

In [15]:
prefix+str(next_num)

'maksssksksss853'

In [16]:
for name, group in data_1_with_group_by_name:
    filename = prefix+str(next_num)    
    create_xml(store_path,path_im1_with,filename,name,group) 
    next_num = next_num+1

data/data1/with_mask/
0-with-mask.jpg
data/data1/with_mask/
1-with-mask.jpg
data/data1/with_mask/
10-with-mask.jpg
data/data1/with_mask/
100-with-mask.jpg
data/data1/with_mask/
101-with-mask.jpg
data/data1/with_mask/
103-with-mask.jpg
data/data1/with_mask/
104-with-mask.jpg
data/data1/with_mask/
105-with-mask.jpg
data/data1/with_mask/
106-with-mask.jpg
data/data1/with_mask/
107-with-mask.jpg
data/data1/with_mask/
108-with-mask.jpg
data/data1/with_mask/
109-with-mask.jpg
data/data1/with_mask/
11-with-mask.jpg
data/data1/with_mask/
110-with-mask.jpg
data/data1/with_mask/
111-with-mask.jpg
data/data1/with_mask/
112-with-mask.jpg
data/data1/with_mask/
113-with-mask.jpg
data/data1/with_mask/
114-with-mask.jpg
data/data1/with_mask/
115-with-mask.jpg
data/data1/with_mask/
117-with-mask.jpg
data/data1/with_mask/
118-with-mask.jpg
data/data1/with_mask/
119-with-mask.jpg
data/data1/with_mask/
12-with-mask.jpg
data/data1/with_mask/
120-with-mask.jpg
data/data1/with_mask/
121-with-mask.jpg
data/da

data/data1/with_mask/
335-with-mask.jpg
data/data1/with_mask/
336-with-mask.jpg
data/data1/with_mask/
338-with-mask.jpg
data/data1/with_mask/
339-with-mask.jpg
data/data1/with_mask/
34-with-mask.jpg
data/data1/with_mask/
340-with-mask.jpg
data/data1/with_mask/
342-with-mask.jpg
data/data1/with_mask/
343-with-mask.jpg
data/data1/with_mask/
344-with-mask.jpg
data/data1/with_mask/
345-with-mask.jpg
data/data1/with_mask/
346-with-mask.jpg
data/data1/with_mask/
347-with-mask.jpg
data/data1/with_mask/
348-with-mask.jpg
data/data1/with_mask/
349-with-mask.jpg
data/data1/with_mask/
35-with-mask.jpg
data/data1/with_mask/
351-with-mask.jpg
data/data1/with_mask/
353-with-mask.jpg
data/data1/with_mask/
355-with-mask.jpg
data/data1/with_mask/
356-with-mask.jpg
data/data1/with_mask/
357-with-mask.jpg
data/data1/with_mask/
358-with-mask.jpg
data/data1/with_mask/
359-with-mask.jpg
data/data1/with_mask/
36-with-mask.jpg
data/data1/with_mask/
362-with-mask.jpg
data/data1/with_mask/
364-with-mask.jpg
dat

data/data1/with_mask/
augmented_image_128.jpg
data/data1/with_mask/
augmented_image_129.jpg
data/data1/with_mask/
augmented_image_13.jpg
data/data1/with_mask/
augmented_image_130.jpg
data/data1/with_mask/
augmented_image_131.jpg
data/data1/with_mask/
augmented_image_132.jpg
data/data1/with_mask/
augmented_image_133.jpg
data/data1/with_mask/
augmented_image_134.jpg
data/data1/with_mask/
augmented_image_135.jpg
data/data1/with_mask/
augmented_image_136.jpg
data/data1/with_mask/
augmented_image_138.jpg
data/data1/with_mask/
augmented_image_139.jpg
data/data1/with_mask/
augmented_image_14.jpg
data/data1/with_mask/
augmented_image_140.jpg
data/data1/with_mask/
augmented_image_142.jpg
data/data1/with_mask/
augmented_image_143.jpg
data/data1/with_mask/
augmented_image_144.jpg
data/data1/with_mask/
augmented_image_145.jpg
data/data1/with_mask/
augmented_image_147.jpg
data/data1/with_mask/
augmented_image_148.jpg
data/data1/with_mask/
augmented_image_149.jpg
data/data1/with_mask/
augmented_imag

data/data1/with_mask/
augmented_image_60.jpg
data/data1/with_mask/
augmented_image_61.jpg
data/data1/with_mask/
augmented_image_62.jpg
data/data1/with_mask/
augmented_image_63.jpg
data/data1/with_mask/
augmented_image_64.jpg
data/data1/with_mask/
augmented_image_65.jpg
data/data1/with_mask/
augmented_image_66.jpg
data/data1/with_mask/
augmented_image_67.jpg
data/data1/with_mask/
augmented_image_68.jpg
data/data1/with_mask/
augmented_image_69.jpg
data/data1/with_mask/
augmented_image_7.jpg
data/data1/with_mask/
augmented_image_70.jpg
data/data1/with_mask/
augmented_image_71.jpg
data/data1/with_mask/
augmented_image_73.jpg
data/data1/with_mask/
augmented_image_74.jpg
data/data1/with_mask/
augmented_image_75.jpg
data/data1/with_mask/
augmented_image_76.jpg
data/data1/with_mask/
augmented_image_77.jpg
data/data1/with_mask/
augmented_image_78.jpg
data/data1/with_mask/
augmented_image_79.jpg
data/data1/with_mask/
augmented_image_8.jpg
data/data1/with_mask/
augmented_image_80.jpg
data/data1/w

In [17]:
for name, group in data_1_without_group_by_name:
    filename = prefix+str(next_num)  
    create_xml(store_path,path_im1_without,filename,name,group)
    next_num = next_num+1

data/data1/without_mask/
0.jpg
data/data1/without_mask/
1.jpg
data/data1/without_mask/
10.jpg
data/data1/without_mask/
100.jpg
data/data1/without_mask/
101.jpg
data/data1/without_mask/
102.jpg
data/data1/without_mask/
104.jpg
data/data1/without_mask/
105.jpg
data/data1/without_mask/
106.jpg
data/data1/without_mask/
107.jpg
data/data1/without_mask/
108.jpg
data/data1/without_mask/
109.jpg
data/data1/without_mask/
11.jpg
data/data1/without_mask/
110.jpg
data/data1/without_mask/
111.jpg
data/data1/without_mask/
112.jpg
data/data1/without_mask/
114.jpg
data/data1/without_mask/
115.jpg
data/data1/without_mask/
116.jpg
data/data1/without_mask/
117.jpg
data/data1/without_mask/
118.jpg
data/data1/without_mask/
119.jpg
data/data1/without_mask/
12.jpg
data/data1/without_mask/
120.jpg
data/data1/without_mask/
122.jpg
data/data1/without_mask/
123.jpg
data/data1/without_mask/
124.jpg
data/data1/without_mask/
125.jpg
data/data1/without_mask/
127.jpg
data/data1/without_mask/
128.jpg
data/data1/withou

data/data1/without_mask/
389.jpg
data/data1/without_mask/
39.jpg
data/data1/without_mask/
390.jpg
data/data1/without_mask/
391.jpg
data/data1/without_mask/
392.jpg
data/data1/without_mask/
393.jpg
data/data1/without_mask/
394.jpg
data/data1/without_mask/
395.jpg
data/data1/without_mask/
397.jpg
data/data1/without_mask/
398.jpg
data/data1/without_mask/
399.jpg
data/data1/without_mask/
4.jpg
data/data1/without_mask/
40.jpg
data/data1/without_mask/
400.jpg
data/data1/without_mask/
401.jpg
data/data1/without_mask/
402.jpg
data/data1/without_mask/
403.jpg
data/data1/without_mask/
404.jpg
data/data1/without_mask/
405.jpg
data/data1/without_mask/
406.jpg
data/data1/without_mask/
407.jpg
data/data1/without_mask/
408.jpg
data/data1/without_mask/
409.jpg
data/data1/without_mask/
410.jpg
data/data1/without_mask/
411.jpg
data/data1/without_mask/
412.jpg
data/data1/without_mask/
413.jpg
data/data1/without_mask/
414.jpg
data/data1/without_mask/
415.jpg
data/data1/without_mask/
416.jpg
data/data1/wit

data/data1/without_mask/
augmented_image_182.jpg
data/data1/without_mask/
augmented_image_183.jpg
data/data1/without_mask/
augmented_image_184.jpg
data/data1/without_mask/
augmented_image_185.jpg
data/data1/without_mask/
augmented_image_186.jpg
data/data1/without_mask/
augmented_image_187.jpg
data/data1/without_mask/
augmented_image_188.jpg
data/data1/without_mask/
augmented_image_189.jpg
data/data1/without_mask/
augmented_image_19.jpg
data/data1/without_mask/
augmented_image_191.jpg
data/data1/without_mask/
augmented_image_192.jpg
data/data1/without_mask/
augmented_image_193.jpg
data/data1/without_mask/
augmented_image_194.jpg
data/data1/without_mask/
augmented_image_196.jpg
data/data1/without_mask/
augmented_image_197.jpg
data/data1/without_mask/
augmented_image_198.jpg
data/data1/without_mask/
augmented_image_199.jpg
data/data1/without_mask/
augmented_image_2.jpg
data/data1/without_mask/
augmented_image_20.jpg
data/data1/without_mask/
augmented_image_200.jpg
data/data1/without_mask/

data/data1/without_mask/
augmented_image_81.jpg
data/data1/without_mask/
augmented_image_82.jpg
data/data1/without_mask/
augmented_image_83.jpg
data/data1/without_mask/
augmented_image_84.jpg
data/data1/without_mask/
augmented_image_85.jpg
data/data1/without_mask/
augmented_image_86.jpg
data/data1/without_mask/
augmented_image_87.jpg
data/data1/without_mask/
augmented_image_88.jpg
data/data1/without_mask/
augmented_image_89.jpg
data/data1/without_mask/
augmented_image_90.jpg
data/data1/without_mask/
augmented_image_91.jpg
data/data1/without_mask/
augmented_image_92.jpg
data/data1/without_mask/
augmented_image_93.jpg
data/data1/without_mask/
augmented_image_94.jpg
data/data1/without_mask/
augmented_image_95.jpg
data/data1/without_mask/
augmented_image_96.jpg
data/data1/without_mask/
augmented_image_97.jpg
data/data1/without_mask/
augmented_image_98.jpg
data/data1/without_mask/
augmented_image_99.jpg


In [18]:
for name, group in data_3_group_by_name:
    filename = prefix+str(next_num)  
    create_xml(store_path,path_data3,filename,name,group)
    next_num = next_num+1

data/data3/
1801.jpg
data/data3/
1802.jpg
data/data3/
1803.jpg
data/data3/
1804.jpg
data/data3/
1805.jpg
data/data3/
1806.jpg
data/data3/
1808.jpg
data/data3/
1809.jpg
data/data3/
1810.jpg
data/data3/
1811.jpg
data/data3/
1812.jpg
data/data3/
1813.jpg
data/data3/
1814.jpg
data/data3/
1815.jpg
data/data3/
1818.jpg
data/data3/
1819.jpg
data/data3/
1821.jpg
data/data3/
1822.jpg
data/data3/
1823.jpg
data/data3/
1825.jpg
data/data3/
1826.jpg
data/data3/
1827.jpg
data/data3/
1828.jpg
data/data3/
1829.jpg
data/data3/
1831.jpg
data/data3/
1833.jpg
data/data3/
1835.jpg
data/data3/
1836.jpg
data/data3/
1838.jpg
data/data3/
1839.jpg
data/data3/
1840.jpg
data/data3/
1841.jpg
data/data3/
1843.jpg
data/data3/
1844.jpg
data/data3/
1845.jpg
data/data3/
1847.jpg
data/data3/
1848.jpg
data/data3/
1849.jpg
data/data3/
1850.jpg
data/data3/
1851.jpg
data/data3/
1852.jpg
data/data3/
1854.jpg
data/data3/
1856.jpg
data/data3/
1857.jpg
data/data3/
1858.jpg
data/data3/
1864.jpg
data/data3/
1867.jpg
data/data3/
1

data/data3/
2340.png
data/data3/
2341.png
data/data3/
2342.png
data/data3/
2344.png
data/data3/
2345.png
data/data3/
2350.png
data/data3/
2351.png
data/data3/
2352.png
data/data3/
2353.png
data/data3/
2354.png
data/data3/
2355.png
data/data3/
2356.png
data/data3/
2357.png
data/data3/
2358.png
data/data3/
2359.png
data/data3/
2360.png
data/data3/
2361.png
data/data3/
2362.png
data/data3/
2363.png
data/data3/
2365.png
data/data3/
2366.png
data/data3/
2367.png
data/data3/
2368.png
data/data3/
2369.png
data/data3/
2371.png
data/data3/
2372.png
data/data3/
2373.png
data/data3/
2375.png
data/data3/
2376.png
data/data3/
2381.png
data/data3/
2382.png
data/data3/
2383.png
data/data3/
2385.png
data/data3/
2386.png
data/data3/
2387.png
data/data3/
2388.png
data/data3/
2389.png
data/data3/
2393.png
data/data3/
2394.png
data/data3/
2395.png
data/data3/
2396.png
data/data3/
2397.png
data/data3/
2399.png
data/data3/
2400.png
data/data3/
2401.png
data/data3/
2403.png
data/data3/
2404.png
data/data3/
2

data/data3/
2890.png
data/data3/
2891.png
data/data3/
2892.png
data/data3/
2893.png
data/data3/
2894.png
data/data3/
2895.png
data/data3/
2896.png
data/data3/
2897.png
data/data3/
2898.png
data/data3/
2899.png
data/data3/
2900.png
data/data3/
2901.png
data/data3/
2902.png
data/data3/
2903.png
data/data3/
2904.png
data/data3/
2905.png
data/data3/
2906.png
data/data3/
2907.png
data/data3/
2908.png
data/data3/
2910.png
data/data3/
2911.png
data/data3/
2912.png
data/data3/
2913.png
data/data3/
2915.png
data/data3/
2916.png
data/data3/
2917.png
data/data3/
2918.png
data/data3/
2925.png
data/data3/
2926.png
data/data3/
2927.png
data/data3/
2928.png
data/data3/
2929.png
data/data3/
2930.png
data/data3/
2933.png
data/data3/
2936.png
data/data3/
2938.png
data/data3/
2939.png
data/data3/
2940.png
data/data3/
2941.png
data/data3/
2943.png
data/data3/
2944.png
data/data3/
2945.png
data/data3/
2946.png
data/data3/
2947.png
data/data3/
2948.png
data/data3/
2949.png
data/data3/
2950.png
data/data3/
2

data/data3/
3423.png
data/data3/
3424.png
data/data3/
3426.png
data/data3/
3427.png
data/data3/
3428.png
data/data3/
3429.png
data/data3/
3430.png
data/data3/
3435.png
data/data3/
3437.png
data/data3/
3438.png
data/data3/
3439.png
data/data3/
3440.png
data/data3/
3441.png
data/data3/
3442.png
data/data3/
3443.png
data/data3/
3444.png
data/data3/
3445.png
data/data3/
3446.png
data/data3/
3447.png
data/data3/
3449.png
data/data3/
3450.png
data/data3/
3451.png
data/data3/
3452.png
data/data3/
3453.png
data/data3/
3454.png
data/data3/
3455.png
data/data3/
3457.png
data/data3/
3458.png
data/data3/
3459.png
data/data3/
3460.png
data/data3/
3462.png
data/data3/
3463.png
data/data3/
3464.png
data/data3/
3465.png
data/data3/
3466.png
data/data3/
3467.png
data/data3/
3468.png
data/data3/
3469.png
data/data3/
3470.png
data/data3/
3471.png
data/data3/
3472.png
data/data3/
3473.png
data/data3/
3474.png
data/data3/
3476.png
data/data3/
3477.png
data/data3/
3478.png
data/data3/
3479.png
data/data3/
3

data/data3/
3925.png
data/data3/
3926.png
data/data3/
3927.png
data/data3/
3928.png
data/data3/
3929.png
data/data3/
3930.png
data/data3/
3931.png
data/data3/
3932.png
data/data3/
3933.png
data/data3/
3934.png
data/data3/
3935.png
data/data3/
3936.png
data/data3/
3937.png
data/data3/
3938.png
data/data3/
3939.png
data/data3/
3940.png
data/data3/
3941.png
data/data3/
3942.png
data/data3/
3943.png
data/data3/
3944.png
data/data3/
3945.png
data/data3/
3946.png
data/data3/
3947.png
data/data3/
3948.png
data/data3/
3950.png
data/data3/
3951.png
data/data3/
3952.png
data/data3/
3953.png
data/data3/
3954.png
data/data3/
3955.png
data/data3/
3956.png
data/data3/
3957.png
data/data3/
3958.png
data/data3/
3959.png
data/data3/
3961.png
data/data3/
3963.png
data/data3/
3964.png
data/data3/
3965.png
data/data3/
3966.png
data/data3/
3967.png
data/data3/
3968.png
data/data3/
3969.png
data/data3/
3971.png
data/data3/
3972.png
data/data3/
3973.png
data/data3/
3974.png
data/data3/
3975.png
data/data3/
3

data/data3/
4395.png
data/data3/
4397.png
data/data3/
4398.png
data/data3/
4399.png
data/data3/
4400.png
data/data3/
4401.png
data/data3/
4403.png
data/data3/
4404.png
data/data3/
4405.png
data/data3/
4406.png
data/data3/
4407.png
data/data3/
4409.png
data/data3/
4410.png
data/data3/
4411.png
data/data3/
4412.png
data/data3/
4413.png
data/data3/
4414.png
data/data3/
4415.png
data/data3/
4417.png
data/data3/
4418.png
data/data3/
4419.png
data/data3/
4420.png
data/data3/
4421.png
data/data3/
4422.png
data/data3/
4423.png
data/data3/
4424.png
data/data3/
4426.png
data/data3/
4427.png
data/data3/
4428.png
data/data3/
4429.png
data/data3/
4430.png
data/data3/
4431.png
data/data3/
4432.png
data/data3/
4433.png
data/data3/
4436.png
data/data3/
4437.png
data/data3/
4439.png
data/data3/
4444.png
data/data3/
4445.png
data/data3/
4446.png
data/data3/
4447.png
data/data3/
4448.png
data/data3/
4450.png
data/data3/
4451.png
data/data3/
4452.png
data/data3/
4453.png
data/data3/
4455.png
data/data3/
4

data/data3/
4908.png
data/data3/
4909.png
data/data3/
4910.png
data/data3/
4911.png
data/data3/
4912.png
data/data3/
4913.png
data/data3/
4915.png
data/data3/
4917.png
data/data3/
4919.png
data/data3/
4920.png
data/data3/
4921.png
data/data3/
4922.png
data/data3/
4923.png
data/data3/
4924.png
data/data3/
4925.png
data/data3/
4926.png
data/data3/
4929.png
data/data3/
4931.png
data/data3/
4932.png
data/data3/
4933.png
data/data3/
4934.png
data/data3/
4935.png
data/data3/
4936.png
data/data3/
4937.png
data/data3/
4938.png
data/data3/
4939.png
data/data3/
4940.png
data/data3/
4941.png
data/data3/
4942.png
data/data3/
4943.png
data/data3/
4944.png
data/data3/
4945.png
data/data3/
4946.png
data/data3/
4948.png
data/data3/
4949.png
data/data3/
4950.png
data/data3/
4952.png
data/data3/
4953.png
data/data3/
4954.png
data/data3/
4965.png
data/data3/
4966.png
data/data3/
4967.png
data/data3/
4969.png
data/data3/
4970.png
data/data3/
4971.png
data/data3/
4972.png
data/data3/
4975.png
data/data3/
4

data/data3/
5530.jpg
data/data3/
5533.jpg
data/data3/
5534.jpg
data/data3/
5536.jpg
data/data3/
5538.jpg
data/data3/
5539.jpg
data/data3/
5540.jpg
data/data3/
5541.jpg
data/data3/
5544.jpg
data/data3/
5545.jpg
data/data3/
5547.jpg
data/data3/
5548.jpg
data/data3/
5550.jpg
data/data3/
5551.jpg
data/data3/
5552.jpg
data/data3/
5555.jpg
data/data3/
5556.jpg
data/data3/
5559.jpg
data/data3/
5560.jpg
data/data3/
5561.jpg
data/data3/
5563.jpg
data/data3/
5564.jpeg
data/data3/
5569.jpg
data/data3/
5570.jpg
data/data3/
5573.jpg
data/data3/
5574.jpg
data/data3/
5575.jpg
data/data3/
5578.jpg
data/data3/
5581.jpg
data/data3/
5582.jpg
data/data3/
5584.jpg
data/data3/
5585.jpg
data/data3/
5586.jpg
data/data3/
5587.jpg
data/data3/
5591.jpg
data/data3/
5592.jpg
data/data3/
5593.jpg
data/data3/
5594.jpg
data/data3/
5600.jpg
data/data3/
5601.jpg
data/data3/
5603.jpg
data/data3/
5604.jpg
data/data3/
5607.jpg
data/data3/
5608.jpg
data/data3/
5609.jpg
data/data3/
5610.jpg
data/data3/
5612.jpg
data/data3/


data/data3/
6182.jpg
data/data3/
6183.jpg
data/data3/
6184.jpg
data/data3/
6185.jpg
data/data3/
6186.jpg
data/data3/
6187.jpg
data/data3/
6188.jpg
data/data3/
6189.jpg
data/data3/
6190.jpg
data/data3/
6191.jpg
data/data3/
6193.jpg
data/data3/
6194.jpg
data/data3/
6195.jpg
data/data3/
6196.jpg
data/data3/
6197.jpg
data/data3/
6198.jpg
data/data3/
6200.jpeg
data/data3/
6200.jpg
data/data3/
6201.jpg
data/data3/
6203.jpg
data/data3/
6204.jpg
data/data3/
6205.jpg
data/data3/
6206.jpg
data/data3/
6207.jpg
data/data3/
6208.jpg
data/data3/
6209.jpg
data/data3/
6210.jpg
data/data3/
6211.jpg
data/data3/
6213.jpg
data/data3/
6214.jpg
data/data3/
6216.jpg
data/data3/
6217.jpg
data/data3/
6218.jpg
data/data3/
6219.jpg
data/data3/
6220.jpg
data/data3/
6221.jpg
data/data3/
6223.jpg
data/data3/
6224.jpg
data/data3/
6225.jpg
data/data3/
6226.jpg
data/data3/
6227.jpg
data/data3/
6228.jpg
data/data3/
6229.jpg
data/data3/
6230.jpeg
data/data3/
6231.jpg
data/data3/
6232.jpg
data/data3/
6233.jpg
data/data3/

In [20]:
store_path = "data/data2"
images = [f for f in listdir(store_path+"/images") if isfile(join(store_path+"/images", f))]
total = len(images)
total

5538

In [21]:
num_train = math.ceil(total*.9)
num_test = total - num_train
images = [f for f in listdir(store_path+"/images") if isfile(join(store_path+"/images", f))]
train = random.sample(images, num_train)
test = [image for image in images if image not in train]
print(len(train))
print(len(test))

4985
553


In [22]:
def make_new_voc(voc_file,new_image_name,output_path):
    new_voc_file = new_image_name[:-3]+'xml'
    tree = ET.parse(voc_file) 
    root = tree.getroot()
    root[1].text = new_image_name
    tree.write(output_path+new_voc_file)

In [23]:
for i,file in enumerate(train):
    new_image_name = file[:12]+str(i)+'.png'
    voc_name = file[:-3]+'xml'
    image = cv2.imread(os.path.join(store_path+"/images",file))
    voc_file = os.path.join(store_path+"/annotations",voc_name)
    make_new_voc(voc_file,new_image_name,store_path+"/random_sample/training/annotations/")
    cv2.imwrite(store_path+"/random_sample/training/images/"+new_image_name, image)

In [24]:
for i,file in enumerate(test):
    new_image_name = file[:12]+str(i+len(train))+'.png'
    voc_name = file[:-3]+'xml'
    image = cv2.imread(os.path.join(store_path+"/images",file))
    voc_file = os.path.join(store_path+"/annotations",voc_name)
    make_new_voc(voc_file,new_image_name,store_path+"/random_sample/testing/annotations/")
    cv2.imwrite(store_path+"/random_sample/testing/images/"+new_image_name, image)

In [ ]:
path_im2="data/data2/images/" 
path_xml2="data/data2/annotations/" 

In [ ]:
def getObjectProperties(filename, objectList):
    objectDict = {}
    for event_type, element in  ET.iterparse(filename, events=('start', 'end')):
        if event_type == 'start' and element.tag == 'filename':
            objectDict['name'] = element.text
        if event_type == 'end' and element.tag == 'name':
            objectDict['classname'] = element.text
        if event_type == 'end' and element.tag == 'xmin':
            objectDict['x1'] = int(element.text)
        if event_type == 'end' and element.tag == 'ymin':
            objectDict['y1'] = int(element.text)
        if event_type == 'end' and element.tag == 'xmax':
            objectDict['x2'] = int(element.text)
        if event_type == 'end' and element.tag == 'ymax':
            objectDict['y2'] = int(element.text)
        if event_type == 'end' and element.tag == 'object':
            objectList.append(dict(objectDict))
    return objectList

In [ ]:
objectList = []
for filename in os.listdir(path_xml2):
    with open(path_xml2+filename) as fd:
        getObjectProperties(fd,objectList)

In [ ]:
data2_df = pd.DataFrame.from_records(objectList)
image2_df = data2_df.apply(lambda row: cv2.cvtColor(cv2.imread(os.path.join(path_im2,row['name'])),cv2.COLOR_BGR2RGB), axis=1)
data2_df = data2_df.merge(image2_df.rename('image'), left_index=True, right_index=True)

data2_df['classname'] = data2_df['classname'].replace(['with_mask','without_mask'],['face_with_mask','face_no_mask'])
print("Generating data2.csv")
data2_df.to_csv('data2.csv')
print("Done")

print("Generating data2.pkl")
with open("data2.pkl", 'wb') as output:
    pickle.dump(data2_df, output, -1)
print("Done")

In [ ]:
#combine all csv
data1 = pd.read_csv("data1.csv",header=0)
data2 = pd.read_csv("data2.csv",header=0)
data3 = pd.read_csv("data3.csv",header=0)
merged_data = pd.concat([data1,data2,data3],ignore_index=True,sort=False)
merged_data.to_csv('all_data.csv')

In [ ]:
merged_data.isnull().values.any()